![fvhfkjv](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTzQ_FBbaBC4xtHoxt0NEdPGBp_2SZSpPGYmw&usqp=CAU)

# استدعاء بعض الملفات المطلوبه 

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# قراءه الملفات 

In [ ]:
df=pd.read_csv('../input/water-potability/water_potability.csv')

# عرض بعض البيانات 

In [ ]:
df.head(10)


In [ ]:
print(df.shape)

# تفقد البيانات لنرى اذا كان يوجد بيانات مفقوده او لا

In [ ]:
df.isnull()

# الان نقوم بتنظيف البيانات المفقوده


In [ ]:
df.dropna( axis=1, inplace=True)
df.head(10)

**الان اصبحت البيانات نظيفه**

**تتمثل الخطوة الأولى في التعرف على بياناتك في اكتشاف الاختلاف
بينما يمكنك وضع أي شيء في قائمة ، تحتوي أعمدة DataFrame على قيم من نوع بيانات معين. عند مقارنة هياكل بيانات Pandas و Python ، سترى أن هذا السلوك يجعل Pandas أسرع بكثير!**

In [ ]:
df.info()

**إظهار إحصائيات الأساسيات
الآن بعد أن تعرفت على أنواع البيانات الموجودة في مجموعة البيانات ، حان الوقت للحصول على نظرة عامة على القيم التي يحتوي عليها كل عمود. يمكنك القيام بذلك باستخدام .describe ():**

In [ ]:
df.describe()

In [ ]:
df['Hardness']

# تصوير البيانات 

In [ ]:
import missingno as msno
msno.matrix(df)

In [ ]:
sns.pairplot(data=df)

In [ ]:
fig, ax = plt.subplots(1,2)
sns.distplot(df, ax=ax[0])
ax[0].set_title("Original Data")
sns.distplot(df, ax=ax[1])
ax[1].set_title("Scaled data")

In [ ]:
df.plot.line(title='df Dataset')

In [ ]:
df['Hardness'].plot.hist()

In [ ]:
df['Solids'].plot.hist()

In [ ]:
df['Chloramines'].plot.hist()

In [ ]:
df['Conductivity'].plot.hist()

In [ ]:
plt.figure(figsize=(7,4)) 
sns.heatmap(df.corr(),annot=True,cmap='cubehelix_r') #draws  heatmap with input as the correlation matrix calculted by(iris.corr())
plt.show()

In [ ]:
sns.distplot(df['Hardness'], color = 'r', label = 'Solids')

In [ ]:
import matplotlib as mpl
%matplotlib inline

df.hist(figsize=(18,10))
plt.show()

In [ ]:
plt.hist(df["Solids"], bins = 30)

In [ ]:
plt.hist(df["Hardness"], bins = 30)

In [ ]:
sns.scatterplot(x='Hardness', y='Solids', data=df)

In [ ]:
sns.lineplot(data=df)

In [ ]:
a = 4  # number of rows
b = 4  # number of columns
c = 1  # initialize plot counter
fig = plt.figure(figsize=(35,32))
for i in df:
    if df[i].dtype=='float64':
        plt.subplot(a, b, c)
        sns.distplot(df[i])
        c = c+1
    else:
        continue
plt.tight_layout()
plt.show() 

In [ ]:
df.head(0)

In [ ]:
#Soure: https://www.kaggle.com/momincks/rain-tomorrow-in-aus-by-xgboostclassifier : Do check him out.

df_dateplot = df.iloc[-950:,:]
plt.figure(figsize=[20,5])
plt.plot(df['Hardness'],df['Solids'],color='blue',linewidth=1, label= 'Outcome')
plt.plot(df['Hardness'],df['Solids'],color='red',linewidth=1, label= 'Outcome')
plt.fill_between(df['Potability'],df['Potability'],['Potability'], facecolor = '#EBF78F')
plt.title('Hardness and Solids')
plt.legend(loc='lower left', frameon=False)
plt.show()

# منحنى ROC مع الانحدار اللوجستي


In [ ]:
# abnormal = 1 and normal = 0
df ['Hardness'] = [1 if i == 'Solids' else 0 for i in df.loc[:,'Hardness']]
x,y = df.loc[:,(df.columns != 'Solids') & (df.columns != 'Age')], df.loc[:,'Potability']

# HYPERPARAMETER TUNING

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
grid = {'n_neighbors': np.arange(1,50)}
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, grid, cv=3) # GridSearchCV
knn_cv.fit(x,y)# Fit

# Print hyperparameter
print("Tuned hyperparameter k: {}".format(knn_cv.best_params_)) 
print("Best score: {}".format(knn_cv.best_score_))

# SVM, pre-process and pipeline

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
steps = [('scalar', StandardScaler()),
         ('SVM', SVC())]
pipeline = Pipeline(steps)
parameters = {'SVM__C':[1, 10, 100],
              'SVM__gamma':[0.1, 0.01]}
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state = 1)
cv = GridSearchCV(pipeline,param_grid=parameters,cv=3)
cv.fit(x_train,y_train)

y_pred = cv.predict(x_test)

print("Accuracy: {}".format(cv.score(x_test, y_test)))
print("Tuned Model Parameters: {}".format(cv.best_params_))

In [ ]:
from sklearn.linear_model import LinearRegression
X_train = np.array(df.iloc[:, :-1].values)
y_train = np.array(df.iloc[:, 1].values)
X_test = np.array(df.iloc[:, :-1].values)
y_test = np.array(df.iloc[:, 1].values)
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = model.score(X_test, y_test)

plt.plot(X_train, model.predict(X_train), color='green')
plt.show()
print(accuracy)

In [ ]:
from sklearn.cluster import KMeans
K_clusters = range(1,8)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = df[['Hardness']]
X_axis = df[['Solids']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.show()